In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import scanpy as sc
import scvelo as scv
import scarches as sca
import numpy as np
import pandas as pd
from velovi import preprocess_data
import scarches as sca

 captum (see https://github.com/pytorch/captum).
/home/icb/cabright61/anaconda3/envs/thesis/lib/python3.8/site-packages/pytorch_lightning/utilities/seed.py:48: LightningDeprecationWarning: `pytorch_lightning.utilities.seed.seed_everything` has been deprecated in v1.8.0 and will be removed in v1.10.0. Please use `lightning_lite.utilities.seed.seed_everything` instead.
  rank_zero_deprecation(
INFO:lightning_lite.utilities.seed:[rank: 0] Global seed set to 0
/home/icb/cabright61/anaconda3/envs/thesis/lib/python3.8/site-packages/pytorch_lightning/loggers/base.py:24: LightningDeprecationWarning: The `pytorch_lightning.loggers.base.rank_zero_experiment` is deprecated in v1.7 and will be removed in v1.9. Please use `pytorch_lightning.loggers.logger.rank_zero_experiment` instead.
  rank_zero_deprecation(


In [ ]:
#Choose dataset
dataset = "pbmc"

In [3]:
input_adata_file = "data/pbmc/pbmc_10k.h5ad"
output_adata_file = "data/pbmc/pbmc_data_annotations.h5ad"

In [3]:
import scanpy as sc
adata=sc.read(input_adata_file)

In [ ]:
if dataset=="pancreas":
    adata.obs["day"] = adata.obs.day.map(lambda day: day.replace(" (reference)", "").replace(" (query)", ""))
    adata=adata[adata.obs["day"]=="15.5"]

adata.layers["spliced"]=adata.layers["spliced"].todense()
adata.layers["unspliced"]=adata.layers["unspliced"].todense()

In [ ]:
scv.pp.filter_and_normalize(adata, min_shared_counts=20, n_top_genes=2000)
scv.pp.moments(adata, n_pcs=30, n_neighbors=30)
adata = preprocess_data(adata)

In [ ]:
scv.pp.filter_and_normalize(adata, min_shared_counts=30, n_top_genes=2000)
scv.pp.moments(adata, n_pcs=30, n_neighbors=30)

adata = preprocess_data(adata)

In [ ]:
data_path='data/annotations/'
sca.utils.add_annotations(adata, [data_path+'PanglaoDB_markers_27_Mar_2020.gmt',data_path+'reactome.gmt'], min_genes=5, clean=True)
#Remove all genes not present in annotations
adata._inplace_subset_var(adata.varm['I'].sum(1)>0)

#Filter out terms with less than 12 genes 
select_terms = adata.varm['I'].sum(0)>12
adata.uns['terms'] = np.array(adata.uns['terms'])[select_terms].tolist()
adata.varm['I'] = adata.varm['I'][:, select_terms]

In [ ]:
adata.write_h5ad(output_adata_file)

In [ ]:
adata.n_obs

Latent Analysis